In [2]:
from sqlalchemy import create_engine, inspect
import sqlite3
from pandas.io import sql
import subprocess
import pandas as pd

In [3]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')




In [4]:
curs = conn.cursor()

In [5]:
curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('Bookings',), ('Facilities',), ('Members',)]

In [6]:
rs = curs.execute('SELECT * FROM Bookings')

In [7]:
df = pd.DataFrame(rs.fetchall())
names = list(map(lambda x: x[0], curs.description))
names

['bookid', 'facid', 'memid', 'starttime', 'slots']

In [8]:
df.head()

,0,1,2,3,4
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [9]:
rs = curs.execute('SELECT * FROM Members')

In [10]:
df1 = pd.DataFrame(rs.fetchall())
names1 = list(map(lambda x: x[0], curs.description))


In [11]:
rs = curs.execute('SELECT * FROM Facilities')

In [12]:
df2 = pd.DataFrame(rs.fetchall())
names2 = list(map(lambda x: x[0], curs.description))
names2

['facid',
 'name',
 'membercost',
 'guestcost',
 'initialoutlay',
 'monthlymaintenance']

In [13]:
rs = pd.read_sql_query("SELECT DISTINCT m.firstname|| ' '|| m.surname AS name, f.name AS CourtName FROM Members AS m INNER JOIN Bookings AS b ON m.memid = b.memid INNER JOIN Facilities AS f ON b.facid = f.facid WHERE m.memid != 0 AND f.name LIKE 'Tennis Court%' ORDER by name LIMIT 10", conn)
rs

,name,CourtName
0,Anne Baker,Tennis Court 1
1,Anne Baker,Tennis Court 2
2,Burton Tracy,Tennis Court 2
3,Burton Tracy,Tennis Court 1
4,Charles Owen,Tennis Court 1
5,Charles Owen,Tennis Court 2
6,Darren Smith,Tennis Court 2
7,David Farrell,Tennis Court 1
8,David Farrell,Tennis Court 2
9,David Jones,Tennis Court 2


In [14]:
rs1 = pd.read_sql_query("SELECT m.firstname|| ' '|| m.surname AS member_name, f.name AS facility_name, CASE WHEN m.memid != 0 THEN b.slots * f.membercost ELSE f.guestcost * b.slots END AS 'Cost' FROM Facilities as f INNER JOIN Bookings as b USING(facid) INNER JOIN Members as m USING(memid) WHERE Cost > 30 AND b.starttime LIKE '2012-09-14%' ORDER BY Cost DESC LIMIT 10", conn)
rs1

,member_name,facility_name,Cost
0,GUEST GUEST,Massage Room 2,320.0
1,GUEST GUEST,Massage Room 1,160.0
2,GUEST GUEST,Massage Room 1,160.0
3,GUEST GUEST,Massage Room 1,160.0
4,GUEST GUEST,Tennis Court 2,150.0
5,GUEST GUEST,Tennis Court 1,75.0
6,GUEST GUEST,Tennis Court 1,75.0
7,GUEST GUEST,Tennis Court 2,75.0
8,GUEST GUEST,Squash Court,70.0
9,Jemima Farrell,Massage Room 1,39.6


In [16]:
rs1 = pd.read_sql_query("SELECT m.firstname|| ' '|| m.surname AS member_name, f.name AS facility_name, CASE WHEN m.memid != 0 THEN b.slots * f.membercost ELSE f.guestcost * b.slots END AS 'Cost' FROM Facilities as f INNER JOIN Bookings as b USING(facid) INNER JOIN Members as m USING(memid) WHERE Cost > 30 AND b.starttime LIKE '2012-09-14%' ORDER BY Cost DESC LIMIT 10", conn)
rs1

,member_name,facility_name,Cost
0,GUEST GUEST,Massage Room 2,320.0
1,GUEST GUEST,Massage Room 1,160.0
2,GUEST GUEST,Massage Room 1,160.0
3,GUEST GUEST,Massage Room 1,160.0
4,GUEST GUEST,Tennis Court 2,150.0
5,GUEST GUEST,Tennis Court 1,75.0
6,GUEST GUEST,Tennis Court 1,75.0
7,GUEST GUEST,Tennis Court 2,75.0
8,GUEST GUEST,Squash Court,70.0
9,Jemima Farrell,Massage Room 1,39.6


In [ ]:
df.columns = names


In [ ]:
df1.columns = names1

In [ ]:
df2.columns = names2

In [ ]:
df.head()

In [ ]:
df1.head()

In [ ]:
df2.head(20)

In [ ]:

rs2 = pd.read_sql_query("SELECT m.surname, m.firstname, r.surname||' ' ||r.firstname AS Recommender FROM Members AS m LEFT JOIN Members AS r ON m.memid = r.recommendedby ORDEr BY m.surname, m.firstname", conn)
rs2

In [ ]:

rs3  = pd.read_sql_query("SELECT f.name AS facility_name, m.firstname||' '||m.surname AS member_name, b.slots AS Usage FROM Facilities AS f INNER JOIN Bookings AS b Using(facid) INNER JOIN Members AS m Using(memid) WHERE memid != 0 LIMIT 10", conn)
rs3

In [ ]:


rs4 = pd.read_sql_query("SELECT name, strftime('%m', starttime) AS month, COUNT(b.slots) AS Usage FROM Facilities INNER JOIN Bookings AS b USING(facid) WHERE b.memid <> 0 GROUP BY name", conn)
rs4

In [ ]:

rs5 = pd.read_sql_query("SELECT f.name, (guestcost * SUM(k.slots) + membercost * SUM(b.slots)) AS total_revenue FROM Facilities AS f INNER JOIN Bookings AS b ON f.facid = b.facid AND b.memid != 0 INNER JOIN Bookings AS k ON b.facid = k.facid and k.memid = 0    GROUP BY k.slots, b.slots HAVING total_revenue < 1000 Order by total_revenue LIMIT 10", conn)
rs5

In [ ]:
rs6 = pd.read_sql_query("SELECT m.firstname||' '|| m.surname AS member_name, f.name AS facility_name, (SELECT b.slots * f.membercost AS costm FROM Bookings AS b INNER JOIN Facilities AS f WHERE b.memid != 0) as 'mem', (SELECT b.slots * f.guestcost AS costg FROM Bookings AS b INNER JOIN Facilities AS f WHERE b.memid = 0) AS 'g' FROM Members AS m INNER JOIN Bookings AS b ON m.memid = b.memid INNER JOIN Facilities AS f ON b.facid = f.facid ",conn)
rs6

In [ ]:
rs7 = pd.read_sql_query("SELECT m.firstname||' '||m.surname AS member_name, f.name AS facility_name, CASE WHEN b.memid != 0 THEN b.slots * f.membercost ELSE f.guestcost * b.slots END AS Cost FROM Facilities as f INNER JOIN Bookings as b ON f.facid = b.facid INNER JOIN Members AS m ON b.memid = m.memid WHERE b.starttime LIKE '2012-09-14%' AND Cost > 30 ORDER BY Cost DESC", conn)


In [ ]:
pd.read_sql_query("SELECT m.firstname||' '|| m.surname AS member_name, f.name AS facility_name, (SELECT f.guestcost * k.slots + f.membercost * b.slots AS Cost FROM Facilities AS f INNER JOIN Bookings AS b ON f.facid = b.facid AND b.memid != 0 INNER JOIN Bookings AS k ON b.facid = k.facid and k.memid = 0 WHERE b.starttime LIKE '2012-09-14%') AS Cost FROM Members AS m INNER JOIN Bookings as b ON m.memid = b.memid INNER JOIN Facilities AS f ON b.facid = f.facid WHERE Cost > 30 AND b.starttime LIKE '2012-09-14%' ORDER BY Cost DESC", conn)

In [19]:
rs1 = pd.read_sql_query("SELECT m.firstname|| ' '|| m.surname AS member_name, f.name AS facility_name, b.slots AS number_slots  FROM Facilities as f INNER JOIN Bookings as b USING(facid) INNER JOIN Members as m USING(memid) WHERE b.starttime LIKE '2012-09-14%'", conn)
rs1

,member_name,facility_name,number_slots
0,Burton Tracy,Tennis Court 1,3
1,David Pinker,Tennis Court 1,3
2,Gerald Butters,Tennis Court 1,3
3,Tim Rownam,Tennis Court 1,3
4,GUEST GUEST,Tennis Court 1,3
...,...,...,...
57,Tim Rownam,Pool Table,1
58,Florence Bader,Pool Table,1
59,Timothy Baker,Pool Table,2
60,David Jones,Pool Table,1


In [23]:
rs1 = pd.read_sql_query("SELECT m.firstname AS member_name, Cost, facility_name FROM ((SELECT b.slots * f.membercost AS Cost, f.name AS facility_name FROM Facilities AS f INNER JOIN Bookings AS b WHERE b.memid != 0) AS sq1 INNER JOIN (SELECT b.slots * f.guestcost AS Cost FROM Facilities AS f INNER JOIN Bookings AS b WHERE b.memid = 0) AS sq2 ON sq1.memid = sq2.memid)) INNER JOIN Members AS m on sq1.memid = m.memid INNER JOIN Bookings AS b ON m.memid = b.memid WHERE Cost > 30 AND b.starttime LIKE '2012-09-14%' ORDER BY Cost DESC", conn)
rs1

DatabaseError: Execution failed on sql 'SELECT m.firstname AS member_name, Cost, facility_name FROM ((SELECT b.slots * f.membercost AS Cost, f.name AS facility_name FROM Facilities AS f INNER JOIN Bookings AS b WHERE b.memid != 0) AS sq1 INNER JOIN (SELECT b.slots * f.guestcost AS Cost FROM Facilities AS f INNER JOIN Bookings AS b WHERE b.memid = 0) AS sq2 ON sq1.memid = sq2.memid)) INNER JOIN Members AS m on sq1.memid = m.memid INNER JOIN Bookings AS b ON m.memid = b.memid WHERE Cost > 30 AND b.starttime LIKE '2012-09-14%' ORDER BY Cost DESC': near ")": syntax error